In [ ]:
# @title 1. 환경 설정 및 라이브러리 설치 (Cell 1)
"""
이 셀에서는 실습에 필요한 라이브러리들을 설치하고 임포트합니다.
- transformers: Hugging Face 모델 및 파이프라인 사용을 위한 라이브러리
- datasets: KLUE 데이터셋 등 Hugging Face Hub의 데이터셋 로드를 위한 라이브러리
- sentencepiece: KoBART 등 일부 모델에서 사용하는 토크나이저 라이브러리
- accelerate: 모델 로딩 및 분산 처리를 도와주는 라이브러리 (특히 NLLB 모델에 유용)
- torch: PyTorch 라이브러리 (기본 백엔드)
"""
# TODO: 필요한 라이브러리를 설치하는 명령어를 작성하세요. (transformers, datasets, sentencepiece, accelerate, torch)
# !pip install ...

import datasets
from datasets import load_dataset, DatasetDict
from transformers import pipeline
import torch
import pandas as pd # 데이터 확인용

# GPU 사용 가능 여부 확인 및 설정 (Colab에서는 보통 GPU 사용 가능)
device = 0 if torch.cuda.is_available() else -1
print(f"사용 가능한 디바이스: {'GPU' if device == 0 else 'CPU'}")

In [ ]:
# @title 2. 데이터셋 로드 및 준비 (Cell 2)
"""
이 셀에서는 KLUE 데이터셋의 MRC 부분을 로드합니다.
전체 데이터셋은 클 수 있으므로, 실습을 위해 일부 데이터만 선택하여 사용합니다.
'context' 컬럼이 우리가 요약할 원본 기사 내용입니다.
"""
full_dataset = load_dataset("klue", "mrc", split="train")

# 실습을 위해 데이터 일부만 선택 (예: 앞 10개)
num_samples_to_use = 10
klue_mrc_subset = full_dataset.select(range(num_samples_to_use))

print("로드된 데이터셋 정보:")
print(klue_mrc_subset)

print("\n첫 번째 데이터 예시 (context 확인):")
print(klue_mrc_subset[0]['context'])

# 데이터셋 확인을 위해 Pandas DataFrame으로 변환 (선택 사항)
df_check = pd.DataFrame(klue_mrc_subset)
print("\n데이터셋 일부 미리보기 (DataFrame):")
display(df_check.head(3)) # Colab 환경에서는 display()가 표 형태로 보여줍니다.

In [ ]:
# @title 3. 요약 모델 파이프라인 로드 (Cell 3)
"""
이 셀에서는 기사 요약을 위한 KoBART 기반 모델 파이프라인을 로드합니다.
모델: gogamza/kobart-summarization
파이프라인 타입: summarization
"""
summarizer = pipeline(
    task="summarization",
    model="gogamza/kobart-summarization",
    device=device
)
print("요약 파이프라인 로드 완료.")

In [ ]:
# @title 4. 기사 내용 요약 및 데이터셋에 추가 (Cell 4)
"""
이 셀에서는 로드된 요약 파이프라인을 사용하여 데이터셋의 'context' 내용을 요약합니다.
map 함수를 사용하여 데이터셋의 각 샘플에 요약 함수를 적용하고,
결과를 'summary'라는 새로운 컬럼에 저장합니다.
"""
# 요약을 수행하는 함수 정의
def summarize_context(example):
  """데이터셋의 'context'를 받아 요약 결과를 반환하는 함수"""
  summary_result = summarizer(
        example['context'],
        max_length=150,
        min_length=30,
        do_sample=False  # deterministic 출력을 원하면 False, 다양성을 원하면 True
    )
    # 결과에서 요약 텍스트 추출
  example['summary'] = summary_result[0]['summary_text']
  return example

print("요약 작업을 시작합니다... (데이터 양에 따라 시간이 소요될 수 있습니다)")
summarized_dataset = klue_mrc_subset.map(summarize_context)
print("요약 작업 완료.")

print("\n요약이 추가된 데이터셋 정보:")
print(summarized_dataset)

print("\n첫 번째 데이터의 원문(context)과 요약(summary):")
print("--- 원문 (Context) ---")
print(summarized_dataset[0]['context'])
print("\n--- 요약 (Summary) ---")
print(summarized_dataset[0]['summary'])

# 데이터셋 확인 (Pandas)
df_check_summary = pd.DataFrame(summarized_dataset)
print("\n요약 추가 후 데이터셋 미리보기 (DataFrame):")
display(df_check_summary.head(3))

In [ ]:
# @title 5. 번역 모델 파이프라인 로드 (Cell 5)
"""
이 셀에서는 한국어 요약본을 영어로 번역하기 위한 NLLB 모델 파이프라인을 로드합니다.
모델: facebook/nllb-200-distilled-600M
파이프라인 타입: translation
NLLB 모델은 다양한 언어를 지원하며, 언어 코드를 지정해야 합니다.
한국어: kor_Hang, 영어: eng_Latn
"""
translator = pipeline(
    task="translation",
    model="facebook/nllb-200-distilled-600M",
    device=device
)

print("번역 파이프라인 로드 완료.")

# 번역 테스트 (선택 사항)
test_translation = translator(
    "안녕하세요?",
    src_lang="kor_Hang",
    tgt_lang="eng_Latn"
)

In [ ]:
# @title 6. 기사 요약본 영어로 번역 및 데이터셋에 추가 (Cell 6)
"""
이 셀에서는 생성된 'summary' 컬럼의 한국어 텍스트를 영어로 번역합니다.
map 함수를 사용하여 번역 함수를 적용하고,
결과를 'english_summary'라는 새로운 컬럼에 저장합니다.
"""
# 번역을 수행하는 함수 정의
def translate_summary_to_english(example):
  """데이터셋의 'summary'를 받아 영어 번역 결과를 반환하는 함수"""
  translation_result = translator(
        example['summary'],
        src_lang="kor_Hang",
        tgt_lang="eng_Latn",
        max_length=150,
        min_length=30,
        do_sample=False  # deterministic 출력을 원하면 False, 다양성을 원하면 True
    )
    # 결과에서 번역 텍스트 추출
  example['english_summary'] = translation_result[0]['translation_text']
  return example

print("번역 작업을 시작합니다... (모델 크기와 데이터 양에 따라 시간이 많이 소요될 수 있습니다)")
translated_dataset = summarized_dataset.map(translate_summary_to_english)
print("번역 작업 완료.")

print("\n번역이 추가된 데이터셋 정보:")
print(translated_dataset)

print("\n첫 번째 데이터의 한국어 요약(summary)과 영어 번역(english_summary):")
print("--- 한국어 요약 (Summary) ---")
print(translated_dataset[0]['summary'])
print("\n--- 영어 번역 (English Summary) ---")
print(translated_dataset[0]['english_summary'])

# 데이터셋 확인 (Pandas)
df_check_translation = pd.DataFrame(translated_dataset)
print("\n번역 추가 후 데이터셋 미리보기 (DataFrame):")
display(df_check_translation.head(3))

In [ ]:
# @title 7. 감정 분석 모델 파이프라인 로드 (Cell 7)
"""
이 셀에서는 영어 텍스트의 감정을 분석하기 위한 모델 파이프라인을 로드합니다.
모델: SamLowe/roberta-base-go_emotions
파이프라인 타입: text-classification
이 모델은 다중 레이블 감정(분노, 기쁨, 슬픔 등)을 예측할 수 있습니다.
"""
emotion_classifier = pipeline(
    task="text-classification",
    model="SamLowe/roberta-base-go_emotions",
    top_k=1,
    device=device
)
print("감정 분석 파이프라인 로드 완료.")

# 감정 분석 테스트 (선택 사항)
test_emotion = emotion_classifier("I am very happy today!")
print(f"감정 분석 테스트: {test_emotion[0][0]}")

In [ ]:
# @title 8. 영어 번역본 감정 분석 및 데이터셋에 추가 (Cell 8)
"""
이 셀에서는 번역된 'english_summary' 컬럼의 텍스트에 대해 감정 분석을 수행합니다.
map 함수를 사용하여 감정 분석 함수를 적용하고,
가장 확률이 높은 감정 레이블을 'emotion'이라는 새로운 컬럼에 저장합니다.
"""
# 감정 분석을 수행하는 함수 정의
def analyze_emotion(example):
  """데이터셋의 'english_summary'를 받아 감정 분석 결과를 반환하는 함수"""
  emotion_result = emotion_classifier(example['english_summary'])
    # 결과는 [[{'label': '...', 'score': ...}]] 형태 (top_k=1이므로 리스트 길이 1)
  example['emotion'] = emotion_result[0][0]['label']
  return example

print("감정 분석 작업을 시작합니다...")
final_dataset = translated_dataset.map(analyze_emotion)
print("감정 분석 작업 완료.")

print("\n최종 데이터셋 정보:")
print(final_dataset)

print("\n첫 번째 데이터의 영어 번역(english_summary)과 감정(emotion):")
print("--- 영어 번역 (English Summary) ---")
print(final_dataset[0]['english_summary'])
print("\n--- 분석된 감정 (Emotion) ---")
print(final_dataset[0]['emotion'])

# 최종 데이터셋 확인 (Pandas)
df_final = pd.DataFrame(final_dataset)
print("\n최종 데이터셋 미리보기 (DataFrame):")
display(df_final) # 전체 선택된 샘플 표시

In [ ]:
# @title 9. 결과 정리 및 마무리 (Cell 9)
"""
모든 단계를 거쳐 생성된 최종 데이터셋(final_dataset)에는
원본 KLUE-MRC 데이터에 'summary', 'english_summary', 'emotion' 컬럼이 추가되었습니다.
이 데이터를 CSV 파일로 저장하거나 추가 분석에 활용할 수 있습니다.
"""
print("모든 작업이 완료되었습니다.")
print("최종 데이터셋 컬럼:", final_dataset.column_names)

# 최종 결과 확인 (첫 5개 샘플)
for i in range(min(5, len(final_dataset))):
  print(f"\n--- 샘플 {i+1} ---")
  print(f"원문 일부: {final_dataset[i]['context'][:100]}...")
  print(f"요약: {final_dataset[i]['summary']}")
  print(f"영어 번역: {final_dataset[i]['english_summary']}")
  print(f"감정 분석: {final_dataset[i]['emotion']}")

In [ ]:
%ls

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%ls

In [ ]:
%cd drive/MyDrive/git

In [ ]:
%ls

In [ ]:
%cd 3-huggingface/

In [ ]:
!git add .

In [ ]:
!git config --global user.email "eoeldroal@naver.com"
!git config --global user.name "eoeldroal"

In [ ]:
! git commit -m "All Done"

In [ ]:
!git push